# HW 11: Problem 14.1

**AUTHOR:** Nolan MacDonald

# Import Dependencies

In [1]:
import os

# Print the current working directory
print(f"Current Working Directory: {os.getcwd()}")

Current Working Directory: /Users/nmacdonald/projects/ISYE6501/HW11


In [2]:
import pandas as pd
from pulp import *

# Data

## Load and Inspect

In [172]:
# Load the diet data into a DataFrame
path_data = "~/projects/ISYE6501/HW11/data/diet.xls"
diet_data = pd.read_excel(path_data)
diet_data.head(5)

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


Check for missing values. List each column name with the total number of missing values.

In [173]:
# Check for nan values in the diet data
nan_vals = diet_data.isna().sum()
print(nan_vals)

Foods              3
Price/ Serving     3
Serving Size       1
Calories           1
Cholesterol mg     1
Total_Fat g        1
Sodium mg          1
Carbohydrates g    1
Dietary_Fiber g    1
Protein g          1
Vit_A IU           1
Vit_C IU           1
Calcium mg         1
Iron mg            1
dtype: int64


The first 5 values look good (Index 0-4).
Look at the last 5 rows.
We can see that `Foods` are only listed for 64 rows (0-63).

In [174]:
diet_data.tail(5)

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [175]:
print(f"Number of rows in the diet data: {len(diet_data)}")

Number of rows in the diet data: 67


## Data Wrangling

The diet data needs to be parsed since it is not in an adequate format to form a model.

First, keep only the valid data from rows 0 to 63.

In [176]:
# Keep only the 64 data points with no missing values
diet_data_clean = diet_data[0:64]
print(f"Number of rows in the diet data: {len(diet_data_clean)}")
diet_data_clean.head(5)

Number of rows in the diet data: 64


,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


Looking for missing values this time returns 0 since the data was cleaned.

In [177]:
# Check for nan values in the diet data
nan_vals_clean = diet_data_clean.isna().sum()
print(nan_vals_clean)

Foods              0
Price/ Serving     0
Serving Size       0
Calories           0
Cholesterol mg     0
Total_Fat g        0
Sodium mg          0
Carbohydrates g    0
Dietary_Fiber g    0
Protein g          0
Vit_A IU           0
Vit_C IU           0
Calcium mg         0
Iron mg            0
dtype: int64


The last 5 values all have valid data after inspection.

In [178]:
diet_data_clean.tail(5)

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0


# Model

Create an LP problem with `LPProblem(name, sense)`.
The name of the problem is defined to use in the output file, `cheapest_diet.lp`.
The second argument is `sense` of the LP problem objective, which can be defined either as `LPMinimize` (Default) or `LPMaximize`.

In [179]:
# Initialize by creating an LP problem
diet_prob = LpProblem("cheapest_diet", LpMinimize)

In [180]:
# Unique list of foods
foods = []
for each in diet_data_clean["Foods"]:
    foods.append(each)
print(foods)

['Frozen Broccoli', 'Carrots,Raw', 'Celery, Raw', 'Frozen Corn', 'Lettuce,Iceberg,Raw', 'Peppers, Sweet, Raw', 'Potatoes, Baked', 'Tofu', 'Roasted Chicken', 'Spaghetti W/ Sauce', 'Tomato,Red,Ripe,Raw', 'Apple,Raw,W/Skin', 'Banana', 'Grapes', 'Kiwifruit,Raw,Fresh', 'Oranges', 'Bagels', 'Wheat Bread', 'White Bread', 'Oatmeal Cookies', 'Apple Pie', 'Chocolate Chip Cookies', 'Butter,Regular', 'Cheddar Cheese', '3.3% Fat,Whole Milk', '2% Lowfat Milk', 'Skim Milk', 'Poached Eggs', 'Scrambled Eggs', 'Bologna,Turkey', 'Frankfurter, Beef', 'Ham,Sliced,Extralean', 'Kielbasa,Prk', "Cap'N Crunch", 'Cheerios', "Corn Flks, Kellogg'S", "Raisin Brn, Kellg'S", 'Rice Krispies', 'Special K', 'Oatmeal', 'Malt-O-Meal,Choc', 'Pizza W/Pepperoni', 'Taco', 'Hamburger W/Toppings', 'Hotdog, Plain', 'Couscous', 'White Rice', 'Macaroni,Ckd', 'Peanut Butter', 'Pork', 'Sardines in Oil', 'White Tuna in Water', 'Popcorn,Air-Popped', 'Potato Chips,Bbqflvr', 'Pretzels', 'Tortilla Chip', 'Chicknoodl Soup', 'Splt Pea&Hams

A list is generated with all of the food names.
A nested list is also created of all diet data that has been cleaned.

Our objective is to find the cheapest diet.
This means that the problem needs the data for each parameter that has data that
is quantitative in either an integer or float.
This does not include `Foods` or `Serving Size` as these columns contain
non-numerical values.
Create dictionaries with each parameter and the corresponding values.


In [181]:
# Create a nested list of diet data values
list_data = diet_data_clean.values.tolist()

# Create dictionaries using dictionary comprehensions
# "Price/ Serving"
cost_dict = {each[0]: each[1] for each in list_data}
# "Calories"
cals_dict = {each[0]: each[3] for each in list_data}
# "Cholesterol mg"
choles_dict = {each[0]: each[4] for each in list_data}
# "Total_Fat g"
fat_dict = {each[0]: each[5] for each in list_data}
# "Sodium mg"
sod_dict = {each[0]: each[6] for each in list_data}
# "Carbohydrates g"
carb_dict = {each[0]: each[7] for each in list_data}
# "Dietary_Fiber g"
fiber_dict = {each[0]: each[8] for each in list_data}
# "Protein g"
pro_dict = {each[0]: each[9] for each in list_data}
# "Vit_A IU"
VA_dict = {each[0]: each[10] for each in list_data}
# "Vit_C IU"
VC_dict = {each[0]: each[11] for each in list_data}
# "Calcium mg"
calc_dict = {each[0]: each[12] for each in list_data}
# "Iron mg"
iron_dict = {each[0]: each[13] for each in list_data}

cost_dict

{'Frozen Broccoli': 0.16,
 'Carrots,Raw': 0.07,
 'Celery, Raw': 0.04,
 'Frozen Corn': 0.18,
 'Lettuce,Iceberg,Raw': 0.02,
 'Peppers, Sweet, Raw': 0.53,
 'Potatoes, Baked': 0.06,
 'Tofu': 0.31,
 'Roasted Chicken': 0.84,
 'Spaghetti W/ Sauce': 0.78,
 'Tomato,Red,Ripe,Raw': 0.27,
 'Apple,Raw,W/Skin': 0.24,
 'Banana': 0.15,
 'Grapes': 0.32,
 'Kiwifruit,Raw,Fresh': 0.49,
 'Oranges': 0.15,
 'Bagels': 0.16,
 'Wheat Bread': 0.05,
 'White Bread': 0.06,
 'Oatmeal Cookies': 0.09,
 'Apple Pie': 0.16,
 'Chocolate Chip Cookies': 0.03,
 'Butter,Regular': 0.05,
 'Cheddar Cheese': 0.25,
 '3.3% Fat,Whole Milk': 0.16,
 '2% Lowfat Milk': 0.23,
 'Skim Milk': 0.13,
 'Poached Eggs': 0.08,
 'Scrambled Eggs': 0.11,
 'Bologna,Turkey': 0.15,
 'Frankfurter, Beef': 0.27,
 'Ham,Sliced,Extralean': 0.33,
 'Kielbasa,Prk': 0.15,
 "Cap'N Crunch": 0.31,
 'Cheerios': 0.28,
 "Corn Flks, Kellogg'S": 0.28,
 "Raisin Brn, Kellg'S": 0.34,
 'Rice Krispies': 0.32,
 'Special K': 0.38,
 'Oatmeal': 0.82,
 'Malt-O-Meal,Choc': 0.52,
 

In [182]:
# dictionary called ‘food_vars’ is created to contain the referenced Variables
food_vars = LpVariable.dicts("Foods",foods,0)

# objective function is added to ‘basic_prob’ first
diet_prob += lpSum([cost_dict[i]*food_vars[i] for i in foods]), "Total Food Cost"

In [183]:
# Diet constraints
diet_prob += lpSum([cals_dict[i] * food_vars[i] for i in foods]) >= 1500, "Min_Calorie_Requirement"
diet_prob += lpSum([cals_dict[i] * food_vars[i] for i in foods]) <= 2500, "Max_Calorie_Requirement"
diet_prob += lpSum([choles_dict[i] * food_vars[i] for i in foods]) >= 30, "Min_Cholesterol_Requirement"
diet_prob += lpSum([choles_dict[i] * food_vars[i] for i in foods]) <= 240, "Max_Cholesterol_Requirement"
diet_prob += lpSum([fat_dict[i] * food_vars[i] for i in foods]) >= 20, "Min_Fat_Requirement"
diet_prob += lpSum([fat_dict[i] * food_vars[i] for i in foods]) <= 70, "Max_Fat_Requirement"
diet_prob += lpSum([sod_dict[i] * food_vars[i] for i in foods]) >= 800, "Min_Sodium_Requirement"
diet_prob += lpSum([sod_dict[i] * food_vars[i] for i in foods]) <= 2000, "Max_Sodium_Requirement"
diet_prob += lpSum([carb_dict[i] * food_vars[i] for i in foods]) >= 130, "Min_Carb_Requirement"
diet_prob += lpSum([carb_dict[i] * food_vars[i] for i in foods]) <= 450, "Max_Carb_Requirement"
diet_prob += lpSum([fiber_dict[i] * food_vars[i] for i in foods]) >= 125, "Min_Fiber_Requirement"
diet_prob += lpSum([fiber_dict[i] * food_vars[i] for i in foods]) <= 250, "Max_Fiber_Requirement"
diet_prob += lpSum([pro_dict[i] * food_vars[i] for i in foods]) >= 60, "Min_Protein_Requirement"
diet_prob += lpSum([pro_dict[i] * food_vars[i] for i in foods]) <= 100, "Max_Protein_Requirement"
diet_prob += lpSum([VA_dict[i] * food_vars[i] for i in foods]) >= 1000, "Min_VA_Requirement"
diet_prob += lpSum([VA_dict[i] * food_vars[i] for i in foods]) <= 10000, "Max_VA_Requirement"
diet_prob += lpSum([VC_dict[i] * food_vars[i] for i in foods]) >= 400, "Min_VC_Requirement"
diet_prob += lpSum([VC_dict[i] * food_vars[i] for i in foods]) <= 5000, "Max_VC_Requirement"
diet_prob += lpSum([calc_dict[i] * food_vars[i] for i in foods]) >= 700, "Min_Calcium_Requirement"
diet_prob += lpSum([calc_dict[i] * food_vars[i] for i in foods]) <= 1500, "Max_Calcium_Requirement"
diet_prob += lpSum([iron_dict[i] * food_vars[i] for i in foods]) >= 10, "Min_Iron_Requirement"
diet_prob += lpSum([iron_dict[i] * food_vars[i] for i in foods]) <= 40, "Max_Iron_Requirement"

In [ ]:
# write the problem to an LP file
diet_prob.writeLP("cheapest_diet.lp")

# call the solver and check the status
diet_prob.solve()

print("Status:", LpStatus[diet_prob.status])

print(f"\nTotal Cost of the Cheapest Nutritional Diet: ${value(diet_prob.objective):.2f}")

# Print the main foods of the diet
results = {}
for v in diet_prob.variables():
    if v.varValue > 0:
        print(f"{v.name}: {v.varValue}")
        # Save the results in a dictionary
        results[v.name] = v.varValue

# Create a DataFrame from the dictionary
results_df = pd.DataFrame(list(results.items()), columns=['Food', 'Servings'])

results_df

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/nmacdonald/mambaforge3/envs/isye6501/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/qp/vfqpfn_n3zd7_yyg46l4zt4c0000gn/T/c853bfa14db842328d02ed8e82e4aebc-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/qp/vfqpfn_n3zd7_yyg46l4zt4c0000gn/T/c853bfa14db842328d02ed8e82e4aebc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 27 COLUMNS
At line 1286 RHS
At line 1309 BOUNDS
At line 1310 ENDATA
Problem MODEL has 22 rows, 64 columns and 1194 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (0) rows, 64 (0) columns and 1194 (0) elements
0  Obj 0 Primal inf 21.63092 (11)
9  Obj 4.3371168
Optimal - objective value 4.3371168
Optimal objective 4.33711681 - 9 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       

,Food,Servings
0,"Foods_Celery,_Raw",52.643710
1,Foods_Frozen_Broccoli,0.259607
2,"Foods_Lettuce,Iceberg,Raw",63.988506
3,Foods_Oranges,2.292939
4,Foods_Poached_Eggs,0.141844
5,"Foods_Popcorn,Air_Popped",13.869322


```
Celery,_Raw [11]
Frozen_Broccoli [20]
Lettuce,Iceberg,Raw [28]
Oranges [35]
Peanut_Butter [36]
Poached_Eggs [39]
Popcorn,Air_Popped [40]

cost_dict['Frozen Broccoli']
cost_dict['Celery, Raw']
cost_dict['Lettuce,Iceberg,Raw']
cost_dict['Oranges']
cost_dict['Popcorn,Air-Popped']
cost_dict['Peanut Butter']
'Poached Eggs'
```

In [185]:
# Calculate the cost of raw celery
celery = diet_prob.variables()[11]
cost_celery = cost_dict['Celery, Raw'] * celery.varValue
print(f"Cost of {round(celery.varValue)} servings of raw celery: ${cost_celery:.2f}")

# Calculate the cost of frozen broccoli
broccoli = diet_prob.variables()[20]
cost_broccoli = cost_dict['Frozen Broccoli'] * broccoli.varValue
print(f"Cost of {round(broccoli.varValue)} servings of frozen broccoli: ${cost_broccoli:.2f}")

# Calculate the cost of raw iceberg lettuce
lettuce = diet_prob.variables()[28]
cost_lettuce = cost_dict['Lettuce,Iceberg,Raw'] * lettuce.varValue
print(f"Cost of {round(lettuce.varValue)} servings of raw iceberg lettuce: ${cost_lettuce:.2f}")

# Calculate the cost of oranges
oranges = diet_prob.variables()[35]
cost_oranges = cost_dict['Oranges'] * oranges.varValue
print(f"Cost of {round(oranges.varValue)} servings of oranges: ${cost_oranges:.2f}")

# Calculate the cost of peanut butter
pb = diet_prob.variables()[36]
cost_pb = cost_dict['Peanut Butter'] * pb.varValue
print(f"Cost of {pb.varValue} servings of peanut butter: ${cost_pb:.2e}")

# Calculate the cost of poached eggs
eggs = diet_prob.variables()[39]
cost_eggs = cost_dict['Poached Eggs'] * eggs.varValue
print(f"Cost of {eggs.varValue} servings of eggs: ${cost_eggs:.2f}")

# Calculate the cost of air-popped popcorn
popcorn = diet_prob.variables()[40]
cost_popcorn = cost_dict['Popcorn,Air-Popped'] * popcorn.varValue
print(f"Cost of {round(popcorn.varValue)} servings of popcorn: ${cost_popcorn:.2f}")



Cost of 53 servings of raw celery: $2.11
Cost of 0 servings of frozen broccoli: $0.04
Cost of 64 servings of raw iceberg lettuce: $1.28
Cost of 2 servings of oranges: $0.34
Cost of 0.0 servings of peanut butter: $0.00e+00
Cost of 0.14184397 servings of eggs: $0.01
Cost of 14 servings of popcorn: $0.55


# Part 2

For each constraint (a, b, c) add them below the diet constraints in the model above.

In [ ]:
# CONSTRAINT A

# Dictionary called ‘food_vars’ is created to contain the referenced Variables
food_vars = LpVariable.dicts("Foods",foods,0)
# Binary variable if food is selected
food_vars_selected = LpVariable.dicts("Selected",foods,0,1,LpBinary)

# If a food is eaten, must eat at least 0.1 serving
for food in foods:
    diet_prob += food_vars[food] >= 0.1 * food_vars_selected[food]
    
# If any of a food is eaten, its binary variable must be 1

for food in foods:
    diet_prob += food_vars_selected[food] >= food_vars[food]*0.0000001 

In [187]:
# CONSTRAINT B
# Include at most 1 of celery and frozen broccoli 
diet_prob += food_vars_selected['Frozen Broccoli'] + food_vars_selected['Celery, Raw'] <= 1 

In [188]:
# CONSTRAINT C
# At least 3 kinds of meat/poultry/fish/eggs
diet_prob += food_vars_selected['Roasted Chicken'] + food_vars_selected['Poached Eggs'] \
        + food_vars_selected['Scrambled Eggs'] + food_vars_selected['Bologna,Turkey'] \
        + food_vars_selected['Frankfurter, Beef'] + food_vars_selected['Ham,Sliced,Extralean'] \
        + food_vars_selected['Kielbasa,Prk'] + food_vars_selected['Pizza W/Pepperoni'] \
        + food_vars_selected['Hamburger W/Toppings'] \
        + food_vars_selected['Hotdog, Plain'] + food_vars_selected['Pork'] \
        + food_vars_selected['Sardines in Oil'] + food_vars_selected['White Tuna in Water'] \
        + food_vars_selected['Chicknoodl Soup'] + food_vars_selected['Splt Pea&Hamsoup'] \
        + food_vars_selected['Vegetbeef Soup'] + food_vars_selected['Neweng Clamchwd'] \
        + food_vars_selected['New E Clamchwd,W/Mlk'] + food_vars_selected['Beanbacn Soup,W/Watr'] >= 3

In [189]:
# Solve the problem
diet_prob.solve()


# Print Results

print()
print("---------The solution to the diet problem is----------")
for var in diet_prob.variables():
    if var.varValue > 0 and "food_select" not in var.name: # Print non binary variables
        print(str(var.varValue)+" units of "+str(var).replace('Foods_','') )
print()
print("Total cost of food = $%.2f" % value(prob.objective))        

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/nmacdonald/mambaforge3/envs/isye6501/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/qp/vfqpfn_n3zd7_yyg46l4zt4c0000gn/T/53a8b73a39684f1c9ab9015864bb4678-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/qp/vfqpfn_n3zd7_yyg46l4zt4c0000gn/T/53a8b73a39684f1c9ab9015864bb4678-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 157 COLUMNS
Duplicate row C0000000 at line 181 <     X0000001  C0000000   1.212000000000e+02 >
Duplicate row C0000001 at line 182 <     X0000001  C0000001   1.212000000000e+02 >
Duplicate row C0000002 at line 183 <     X0000001  C0000002   1.830000000000e+01 >
Duplicate row C0000003 at line 184 <     X0000001  C0000003   1.830000000000e+01 >
Duplicate row C0000004 at line 185 <     X0000001  C0000004   4.700000000000e+00 >
Duplicate row C0000005 at line 186 <     X0000001  C0000005   4.700

PulpSolverError: Pulp: Error while executing /Users/nmacdonald/mambaforge3/envs/isye6501/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc